**<파일 경로>**
- C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/weather/test

In [1]:
filepath = input()

C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/weather/test


In [2]:
import pandas as pd
import datetime as dt
from tqdm.notebook import tqdm

# 2020년 1~6월 기상 데이터 불러오기

In [3]:
weather_total = pd.read_csv('{}/OBS_AWS_TIM_2020_01~06.csv'.format(filepath), encoding = 'cp949')
weather_total

,지점,지점명,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),현지기압(hPa),해면기압(hPa),습도(%)
0,116,관악(레),2020-01-01 00:00,-8.8,266.9,4.5,0.0,NaN,NaN,97.0
1,116,관악(레),2020-01-01 01:00,-7.7,251.2,4.8,0.0,NaN,NaN,97.9
2,116,관악(레),2020-01-01 02:00,-7.5,265.6,5.2,0.0,NaN,NaN,97.9
3,116,관악(레),2020-01-01 03:00,-7.0,282.3,3.4,0.0,NaN,NaN,97.9
4,116,관악(레),2020-01-01 04:00,-6.6,277.2,3.8,0.0,NaN,NaN,97.9
...,...,...,...,...,...,...,...,...,...,...
121996,889,현충원,2020-06-30 19:00,21.4,320.5,1.3,0.0,995.8,997.7,76.7
121997,889,현충원,2020-06-30 20:00,20.6,312.2,1.8,0.0,996.8,998.7,85.0
121998,889,현충원,2020-06-30 21:00,20.0,352.3,1.4,0.0,997.7,999.6,85.9
121999,889,현충원,2020-06-30 22:00,19.0,266.6,1.1,0.5,998.3,1000.2,96.6


# 필요한 변수들만 추출

In [4]:
weather_total = weather_total[['일시', '지점명', '기온(°C)', '풍속(m/s)', '강수량(mm)', '습도(%)']]
weather_total.head()

,일시,지점명,기온(°C),풍속(m/s),강수량(mm),습도(%)
0,2020-01-01 00:00,관악(레),-8.8,4.5,0.0,97.0
1,2020-01-01 01:00,관악(레),-7.7,4.8,0.0,97.9
2,2020-01-01 02:00,관악(레),-7.5,5.2,0.0,97.9
3,2020-01-01 03:00,관악(레),-7.0,3.4,0.0,97.9
4,2020-01-01 04:00,관악(레),-6.6,3.8,0.0,97.9


# 변수명 변경
- 한글로 되어 있는 변수명을 모두 영어로 변경해주겠다.
    - 일시 = date
    - 지점명 = obs_name
    - 기온(°C) = temp
    - 풍속(m/s) = windspeed
    - 강수량(mm) = rain
    - 습도(%) = humidity

In [5]:
col_name = {
    '일시': 'date', '지점명': 'obs_name',
    '기온(°C)': 'temp', '풍속(m/s)': 'windspeed',
    '강수량(mm)': 'rain', '습도(%)': 'humidity'
}

weather_total.rename(columns = col_name, inplace = True)
weather_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122001 entries, 0 to 122000
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   date       122001 non-null  object 
 1   obs_name   122001 non-null  object 
 2   temp       121643 non-null  float64
 3   windspeed  121511 non-null  float64
 4   rain       121419 non-null  float64
 5   humidity   73827 non-null   float64
dtypes: float64(4), object(2)
memory usage: 5.6+ MB


C:\Users\Brian\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


# 지점명(obs_name) 변수 처리

## 지점명(obs_name) 변수 변경 및 삭제
- 기상 관측 지점명(obs_name) 변수에 대해서 다음과 같은 처리를 해주도록 하겠다.

**<기상 데이터 관측 지점명(obs_name) 변경 및 삭제>**
1. 변경 지점명(obs_name) 리스트
    - 기상청
        - 해당 관측 지점은 주소가 "동작구"이므로, "동작"이라고 변경해주도록 하겠다.
        - 즉, "기상청 >> 동작"
    - 한강
        - 해당 관측 지점은 주소가 "영등포구"이므로, "영등포"라고 변경해주도록 하겠다.
        - 즉, "한강 >> 영등포"
    - 현충원
        - 해당 관측 지점은 주소가 "동작구"이므로, "동작"이라고 변경해주도록 하겠다.
        - 즉, "현충원 >> 동작"
    - 북악산
        - 해당 관측 지점은 주소가 "종로구"이므로, "종로"라고 변경해주도록 하겠다.
        - 즉, "북악산 >> 종로"
2. 삭제 지점명(obs_name) 리스트
    - 관악(레)
        - 해당 관측 지점은 주소가 "경기도"이기 때문에 삭제하도록 하겠다.
    - 남현
        - 해당 관측 지점은 주소가 "전라남도"이기 때문에 삭제하도록 하겠다.

In [6]:
# 지점명 삭제
drop_idx = weather_total[(weather_total['obs_name'] == '관악(레)') | (weather_total['obs_name'] == '남현')].index
weather_total2 = weather_total.drop(drop_idx)

In [7]:
# 지점명 변경
def process_obs_name(x):
    if x == '기상청':
        return '동작'
    elif x == '한강':
        return '영등포'
    elif x == '현충원':
        return '동작'
    elif x == '북악산':
        return '종로'
    else:
        return x

weather_total2['obs_name'] = weather_total2['obs_name'].apply(process_obs_name)

In [8]:
# 서울특별시 자치구 개수 확인
len(set(weather_total2.obs_name))

25

In [9]:
list(set(weather_total2.obs_name))

['성북',
 '관악',
 '용산',
 '구로',
 '영등포 ',
 '양천',
 '강남',
 '중랑',
 '송파',
 '강동',
 '서대문',
 '은평',
 '강북',
 '동작',
 '동대문',
 '노원',
 '영등포',
 '성동',
 '광진',
 '강서',
 '중구',
 '마포',
 '서초',
 '금천',
 '도봉']

- 작업을 수행해 준 뒤에 자치구의 개수가 25개가 맞는지 확인해보니, 25개가 맞긴 하지만 "영등포"구가 2개가 존재하며 "종로"구는 존재하지 않음을 알 수 있다.
- 따라서 먼저 중복되어 들어있는 "영등포"구에 대한 처리부터 진행해주겠다.

In [10]:
weather_total2.obs_name.value_counts()

동작      8730
서초      4368
성북      4368
동대문     4368
송파      4368
노원      4368
강북      4368
영등포     4368
마포      4368
서대문     4368
성동      4368
중구      4368
금천      4368
용산      4368
관악      4368
영등포     4368
도봉      4368
양천      4368
강동      4365
강남      4358
강서      4332
중랑      4326
광진      4320
은평      4312
구로      4295
Name: obs_name, dtype: int64

- 자세히 확인해보니, "영등포" 뒤에 띄어쓰기가 들어간 데이터들이 존재한다.
    - 따라서 이러한 데이터들에 대하여 모두 뒤에 붙은 띄어쓰기를 제거해주도록 하겠다.
    - ex) "영등포 " >> "영등포"

In [11]:
weather_total2[weather_total2['obs_name'] == '영등포']

,date,obs_name,temp,windspeed,rain,humidity
82797,2020-01-01 00:00,영등포,-4.2,1.6,0.0,47.0
82798,2020-01-01 01:00,영등포,-3.9,2.2,0.0,48.1
82799,2020-01-01 02:00,영등포,-4.0,1.9,0.0,49.0
82800,2020-01-01 03:00,영등포,-4.0,2.6,0.0,56.0
82801,2020-01-01 04:00,영등포,-4.1,2.2,0.0,58.5
...,...,...,...,...,...,...
87160,2020-06-30 19:00,영등포,21.3,5.1,0.0,75.5
87161,2020-06-30 20:00,영등포,20.0,4.6,0.0,85.5
87162,2020-06-30 21:00,영등포,19.2,2.5,0.5,87.3
87163,2020-06-30 22:00,영등포,19.3,2.9,0.5,88.2


In [12]:
weather_total2[weather_total2['obs_name'] == '영등포 ']

,date,obs_name,temp,windspeed,rain,humidity
113271,2020-01-01 00:00,영등포,-4.5,0.9,0.0,NaN
113272,2020-01-01 01:00,영등포,-4.1,0.0,0.0,NaN
113273,2020-01-01 02:00,영등포,-4.1,0.1,0.0,NaN
113274,2020-01-01 03:00,영등포,-4.1,0.6,0.0,NaN
113275,2020-01-01 04:00,영등포,-4.0,0.0,0.0,NaN
...,...,...,...,...,...,...
117634,2020-06-30 19:00,영등포,21.2,0.6,0.0,NaN
117635,2020-06-30 20:00,영등포,19.8,1.2,0.0,NaN
117636,2020-06-30 21:00,영등포,18.7,0.1,0.5,NaN
117637,2020-06-30 22:00,영등포,19.1,0.4,0.0,NaN


In [13]:
def edit_obs_name(x):
    if x == '영등포 ':
        return '영등포'
    else:
        return x
    
weather_total2['obs_name'] = weather_total2['obs_name'].apply(edit_obs_name)
weather_total2.obs_name.value_counts()

영등포    8736
동작     8730
서초     4368
중구     4368
동대문    4368
송파     4368
노원     4368
강북     4368
마포     4368
서대문    4368
성북     4368
성동     4368
금천     4368
용산     4368
관악     4368
도봉     4368
양천     4368
강동     4365
강남     4358
강서     4332
중랑     4326
광진     4320
은평     4312
구로     4295
Name: obs_name, dtype: int64

In [14]:
# 서울특별시 자치구 개수 재확인
len(set(weather_total2.obs_name))

24

- 중복되어 있던 "영등포"구를 처리해주니까 총 24개의 자치구가 존재한다고 나온다.
- 이는 "종로"구에 대한 데이터가 존재하지 않기 때문이다.
    - 따라서 "종로"구와 인접한 "성북"구의 데이터를 "종로"구 데이터로 취급하여 사용하도록 하겠다.
    - 이를 위해 "성북"구 데이터를 따로 추출해서 하나의 데이터프레임으로 만든 뒤, 기존의 기상 데이터와 결합시켜주겠다.

In [15]:
# "성북"구 데이터를 "종로"구 데이터로 대체
df_jongro = weather_total2[weather_total2['obs_name'] == '성북']
df_jongro

,date,obs_name,temp,windspeed,rain,humidity
65381,2020-01-01 00:00,성북,-7.8,1.1,0.0,49.7
65382,2020-01-01 01:00,성북,-7.2,0.7,0.0,49.1
65383,2020-01-01 02:00,성북,-6.9,0.7,0.0,51.0
65384,2020-01-01 03:00,성북,-6.9,1.2,0.0,54.7
65385,2020-01-01 04:00,성북,-6.9,0.3,0.0,56.3
...,...,...,...,...,...,...
69744,2020-06-30 19:00,성북,19.6,4.7,0.0,78.1
69745,2020-06-30 20:00,성북,19.3,4.4,0.0,79.6
69746,2020-06-30 21:00,성북,18.0,0.4,1.0,90.8
69747,2020-06-30 22:00,성북,17.7,1.4,0.5,90.5


In [16]:
def make_jongro_data(x):
    if x == '성북':
        return '종로'
    
df_jongro['obs_name'] = df_jongro['obs_name'].apply(make_jongro_data)
df_jongro['obs_name'].value_counts()

<ipython-input-16-02ec3b59958c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jongro['obs_name'] = df_jongro['obs_name'].apply(make_jongro_data)


종로    4368
Name: obs_name, dtype: int64

In [17]:
# 만들어 준 "종로"구 데이터를 기존의 기상 데이터 셋에 결합
weather_total3 = pd.concat([weather_total2, df_jongro], axis = 0, ignore_index = True)
weather_total3

,date,obs_name,temp,windspeed,rain,humidity
0,2020-01-01 00:00,강남,-5.1,1.0,0.0,41.5
1,2020-01-01 01:00,강남,-5.0,0.8,0.0,43.4
2,2020-01-01 02:00,강남,-4.8,1.3,0.0,47.0
3,2020-01-01 03:00,강남,-4.9,0.7,0.0,50.3
4,2020-01-01 04:00,강남,-4.7,1.4,0.0,55.2
...,...,...,...,...,...,...
117657,2020-06-30 19:00,종로,19.6,4.7,0.0,78.1
117658,2020-06-30 20:00,종로,19.3,4.4,0.0,79.6
117659,2020-06-30 21:00,종로,18.0,0.4,1.0,90.8
117660,2020-06-30 22:00,종로,17.7,1.4,0.5,90.5


In [18]:
weather_total3.obs_name.value_counts()

영등포    8736
동작     8730
서초     4368
성북     4368
동대문    4368
송파     4368
종로     4368
노원     4368
강북     4368
마포     4368
서대문    4368
성동     4368
중구     4368
금천     4368
용산     4368
관악     4368
도봉     4368
양천     4368
강동     4365
강남     4358
강서     4332
중랑     4326
광진     4320
은평     4312
구로     4295
Name: obs_name, dtype: int64

In [19]:
# 서울특별시 자치구 개수 재확인
len(set(weather_total3.obs_name))

25

### 중복된 값 제거
- 지점명(obs_name)이 "동작"과 영등포"인 경우, 중복된 값들의 존재로 인해 값이 높게 나온 것을 확인할 수 있다.
- 따라서 중복된 값들을 모두 제거해주도록 하겠다.
    - ```drop_duplicates()``` 함수를 사용해서 "date"와 "obs_name" 변수를 기준으로 중복된 값들을 제거해주겠다.
    - 이때 ```keep = 'first'``` 옵션을 통해 첫 번째 값만 남겨주겠다.

In [20]:
weather_total3.drop_duplicates(['date', 'obs_name'], keep = 'first', inplace = True)
weather_total3.obs_name.value_counts()

서초     4368
중구     4368
동대문    4368
송파     4368
종로     4368
노원     4368
강북     4368
영등포    4368
마포     4368
서대문    4368
성북     4368
성동     4368
금천     4368
용산     4368
관악     4368
동작     4368
도봉     4368
양천     4368
강동     4365
강남     4358
강서     4332
중랑     4326
광진     4320
은평     4312
구로     4295
Name: obs_name, dtype: int64

## 지점명(obs_name)별 모든 일자(date) 존재 여부 확인
- 각 지점명(obs_name)별로 모든 일자(date)가 존재하는지 확인해보고, 그렇지 않다면 존재하지 않는 부분들에 대한 처리를 해주도록 하겠다.

In [21]:
# 각 지점명별로 (중복된 값을 제외하고) 총 4368개의 일자가 존재해야 함
len(set(weather_total3['date'])) # 182(날짜) * 24(시간) = 4368개

4368

- 위에서 ```value_counts()``` 함수를 통해, 지점명(obs_name)이 "강동, 강남, 강서, 중랑, 광진, 은평, 구로"의 경우 모든 일자(date)가 존재하지 않음을 확인할 수 있었다.
    - 즉, 지점명별로 (중복된 값을 제외하고) 총 4368개의 일자가 존재하지 않는다.
- 따라서 이에 대한 처리를 수행해주겠다.

### 각 지점명별로 비어있는 일자(date)를 확인 후, 채워해주는 함수 생성
- 각 지점명별로 비어있는 일자를 찾아내서, 근접한 지점의 값들로 대체해주는 작업을 수행해주도록 하겠다.
    - ex) "강남" 지점의 특정 일자들이 비어있는 경우, 해당 일자의 "송파" 지점 값들로 대체
- 또한 반복 작업을 최소화하기 위해 하나의 함수를 만들어서 사용하겠다.

In [22]:
# 중복 값이 존재하지 않는 일자(date)로 이루어진 DataFrame 생성
date_nodup = pd.DataFrame(weather_total3[weather_total3['obs_name'] == '성동']['date']).reset_index(drop = True)
date_nodup

,date
0,2020-01-01 00:00
1,2020-01-01 01:00
2,2020-01-01 02:00
3,2020-01-01 03:00
4,2020-01-01 04:00
...,...
4363,2020-06-30 19:00
4364,2020-06-30 20:00
4365,2020-06-30 21:00
4366,2020-06-30 22:00


In [23]:
def fill_null_date(null_obs, replace_obs):
    # 지점명이 null_obs로 이루어진 DataFrame 생성
    df_null_obs = weather_total3[weather_total3['obs_name'] == null_obs].reset_index(drop = True)

    # date_nodup과 df_null_obs을 left join
    df_null_check = pd.merge(date_nodup, df_null_obs, how = 'left', on = 'date')
    # 이를 통해 비어있는 일자(date) 확인
    df_null_date = df_null_check[df_null_check['obs_name'].isnull()][['date']]

    # null_obs 지점의 비어있는 일자(date)에 대해서 replace_obs 지점의 관측값들로 대체
    df_replace_obs = weather_total3[weather_total3['obs_name'] == replace_obs].reset_index(drop = True)
    df_null_date2 = pd.merge(df_null_date, df_replace_obs, how = 'left', on = 'date')

    # replace_obs라고 되어 있는 지점명을 null_obs으로 변경
    def change_obs_name(x):
        if x == replace_obs:
            return null_obs

    df_null_date2['obs_name'] = df_null_date2['obs_name'].apply(change_obs_name)

    # 지점명이 null_obs로 이루어진 DataFrame에 비어있는 일자(date)가 채워진 DataFrame을 결합
    df_null_obs2 = pd.concat([df_null_obs, df_null_date2], axis = 0)
    df_null_obs2.sort_values(by = 'date', inplace = True)
    
    return df_null_obs2.reset_index(drop = True)

In [24]:
weather_total3.obs_name.value_counts()

서초     4368
중구     4368
동대문    4368
송파     4368
종로     4368
노원     4368
강북     4368
영등포    4368
마포     4368
서대문    4368
성북     4368
성동     4368
금천     4368
용산     4368
관악     4368
동작     4368
도봉     4368
양천     4368
강동     4365
강남     4358
강서     4332
중랑     4326
광진     4320
은평     4312
구로     4295
Name: obs_name, dtype: int64

In [25]:
# "강동" 지점에 비어있는 일자(date) 처리
obs_강동 = fill_null_date('강동', '송파')
# "강남" 지점에 비어있는 일자(date) 처리
obs_강남 = fill_null_date('강남', '송파')
# "강서" 지점에 비어있는 일자(date) 처리
obs_강서 = fill_null_date('강서', '양천')
# "중랑" 지점에 비어있는 일자(date) 처리
obs_중랑 = fill_null_date('중랑', '동대문')
# "광진" 지점에 비어있는 일자(date) 처리
obs_광진 = fill_null_date('광진', '성동')
# "은평" 지점에 비어있는 일자(date) 처리
obs_은평 = fill_null_date('은평', '성북')
# "구로" 지점에 비어있는 일자(date) 처리
obs_구로 = fill_null_date('구로', '양천')

In [26]:
# 각 지점별로 비어있는 일자(date)가 존재하는지 재확인
check_null_count = [obs_강동.obs_name.isnull().sum(), obs_강남.obs_name.isnull().sum(), obs_강서.obs_name.isnull().sum(),
                    obs_중랑.obs_name.isnull().sum(), obs_광진.obs_name.isnull().sum(), obs_은평.obs_name.isnull().sum(),
                    obs_구로.obs_name.isnull().sum()]
check_null_count

[0, 0, 0, 0, 0, 0, 0]

### 모든 DataFrame을 하나로 결합
- 비어있는 일자(date)와 그에 해당되는 변수 값들을 채워넣은 각 지점명별 DataFrame들과, 일자(date)가 비어있지 않았던 나머지 자치구들의 DataFrame들을 결합해서 하나의 DataFrame으로 만들어주겠다.

In [27]:
obs_도봉 = weather_total3[weather_total3['obs_name'] == '도봉'].reset_index(drop = True)
obs_중구 = weather_total3[weather_total3['obs_name'] == '중구'].reset_index(drop = True)
obs_양천 = weather_total3[weather_total3['obs_name'] == '양천'].reset_index(drop = True)
obs_서대문 = weather_total3[weather_total3['obs_name'] == '서대문'].reset_index(drop = True)
obs_금천 = weather_total3[weather_total3['obs_name'] == '금천'].reset_index(drop = True)
obs_성동 = weather_total3[weather_total3['obs_name'] == '성동'].reset_index(drop = True)
obs_서초 = weather_total3[weather_total3['obs_name'] == '서초'].reset_index(drop = True)
obs_강북 = weather_total3[weather_total3['obs_name'] == '강북'].reset_index(drop = True)
obs_동대문 = weather_total3[weather_total3['obs_name'] == '동대문'].reset_index(drop = True)
obs_성북 = weather_total3[weather_total3['obs_name'] == '성북'].reset_index(drop = True)
obs_송파 = weather_total3[weather_total3['obs_name'] == '송파'].reset_index(drop = True)
obs_관악 = weather_total3[weather_total3['obs_name'] == '관악'].reset_index(drop = True)
obs_종로 = weather_total3[weather_total3['obs_name'] == '종로'].reset_index(drop = True)
obs_마포 = weather_total3[weather_total3['obs_name'] == '마포'].reset_index(drop = True)
obs_용산 = weather_total3[weather_total3['obs_name'] == '용산'].reset_index(drop = True)
obs_노원 = weather_total3[weather_total3['obs_name'] == '노원'].reset_index(drop = True)
obs_영등포 = weather_total3[weather_total3['obs_name'] == '영등포'].reset_index(drop = True)
obs_동작 = weather_total3[weather_total3['obs_name'] == '동작'].reset_index(drop = True)

In [28]:
concat_list = [obs_성동, obs_동작, obs_영등포, obs_송파, obs_강서, obs_은평, obs_금천, obs_중구,
               obs_마포, obs_중랑, obs_용산, obs_강남, obs_동대문, obs_노원, obs_관악, obs_강동,
               obs_서초, obs_성북, obs_양천, obs_강북, obs_구로, obs_도봉, obs_서대문, obs_광진, obs_종로]
obs_total = pd.concat(concat_list, axis = 0, ignore_index = True)
obs_total

,date,obs_name,temp,windspeed,rain,humidity
0,2020-01-01 00:00,성동,-4.4,1.1,0.0,44.6
1,2020-01-01 01:00,성동,-4.2,1.2,0.0,44.9
2,2020-01-01 02:00,성동,-4.0,1.4,0.0,47.8
3,2020-01-01 03:00,성동,-4.1,1.4,0.0,53.8
4,2020-01-01 04:00,성동,-4.1,1.5,0.0,56.6
...,...,...,...,...,...,...
109195,2020-06-30 19:00,종로,19.6,4.7,0.0,78.1
109196,2020-06-30 20:00,종로,19.3,4.4,0.0,79.6
109197,2020-06-30 21:00,종로,18.0,0.4,1.0,90.8
109198,2020-06-30 22:00,종로,17.7,1.4,0.5,90.5


In [29]:
obs_total.obs_name.isnull().sum()

0

In [30]:
obs_total.obs_name.value_counts()

서초     4368
강서     4368
광진     4368
강동     4368
동대문    4368
송파     4368
종로     4368
노원     4368
강북     4368
영등포    4368
마포     4368
서대문    4368
성동     4368
성북     4368
중구     4368
구로     4368
금천     4368
용산     4368
관악     4368
은평     4368
동작     4368
강남     4368
중랑     4368
도봉     4368
양천     4368
Name: obs_name, dtype: int64

# 일시(date) 변수 처리
- date 변수의 타입을 datetime으로 변경하고, "시간대"를 의미하는 hour 변수를 만들어주도록 하겠다.
- 그 다음, date 변수의 형태를 다음과 같이 변경해주겠다.
    - ex) "2018-01-01 05:00:00" >> "2018-01-01"

In [31]:
# 최종 데이터 셋을 구성하기 위해 복사본 사용
weather_final = obs_total.copy()

In [32]:
# datetime 타입으로 변경
weather_final['date'] = pd.to_datetime(weather_final['date'])

# hour 변수 생성
weather_final['hour'] = weather_final['date'].dt.hour
# date 변수 생성(기존의 date 변수에 덮어쓰기)
weather_final['date'] = weather_final['date'].dt.date

In [33]:
weather_final

,date,obs_name,temp,windspeed,rain,humidity,hour
0,2020-01-01,성동,-4.4,1.1,0.0,44.6,0
1,2020-01-01,성동,-4.2,1.2,0.0,44.9,1
2,2020-01-01,성동,-4.0,1.4,0.0,47.8,2
3,2020-01-01,성동,-4.1,1.4,0.0,53.8,3
4,2020-01-01,성동,-4.1,1.5,0.0,56.6,4
...,...,...,...,...,...,...,...
109195,2020-06-30,종로,19.6,4.7,0.0,78.1,19
109196,2020-06-30,종로,19.3,4.4,0.0,79.6,20
109197,2020-06-30,종로,18.0,0.4,1.0,90.8,21
109198,2020-06-30,종로,17.7,1.4,0.5,90.5,22


In [34]:
weather_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109200 entries, 0 to 109199
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   date       109200 non-null  object 
 1   obs_name   109200 non-null  object 
 2   temp       108825 non-null  float64
 3   windspeed  108850 non-null  float64
 4   rain       108704 non-null  float64
 5   humidity   65281 non-null   float64
 6   hour       109200 non-null  int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 5.8+ MB


# 지점명(obs_name) 변수 값 형태 및 변수명 변경
- 추후에 최종 데이터 셋을 만들 때 조인키로 사용하기 위해, "지점명" 변수 값의 형태를 다음과 같이 바꿔주겠다,
    - ex) "강남" >> "강남구"
    - 단, "중구"의 경우 수정 작업이 필요 없다. (이미 자치구명 형태로 입력되어 있기 때문)
        - 따라서 먼저 모든 "지점명" 변수 값의 형태를 변경해 준 뒤, "중구구"라고 변경되었을 "중구" 지점에 대해서만 "구" 문자열 하나를 지워주도록 하겠다.
        - ex) "중구구" >> "중구"
- 그리고 "지점명(obs_name)"이 의미하는 바가 자치구와 동일하므로, 변수명을 "gu"라고 변경해주겠다.

In [35]:
# 변수 값의 형태 변경
weather_final['obs_name'] = weather_final['obs_name'] + '구'

# "중구구" >> "중구" 로 수정
def jung_gu_name(x):
    if x == '중구구':
        return '중구'
    else:
        return x
weather_final['obs_name'] = weather_final['obs_name'].apply(jung_gu_name)

# 변수명 변경
weather_final.rename(columns = {'obs_name': 'gu'}, inplace = True)
weather_final

,date,gu,temp,windspeed,rain,humidity,hour
0,2020-01-01,성동구,-4.4,1.1,0.0,44.6,0
1,2020-01-01,성동구,-4.2,1.2,0.0,44.9,1
2,2020-01-01,성동구,-4.0,1.4,0.0,47.8,2
3,2020-01-01,성동구,-4.1,1.4,0.0,53.8,3
4,2020-01-01,성동구,-4.1,1.5,0.0,56.6,4
...,...,...,...,...,...,...,...
109195,2020-06-30,종로구,19.6,4.7,0.0,78.1,19
109196,2020-06-30,종로구,19.3,4.4,0.0,79.6,20
109197,2020-06-30,종로구,18.0,0.4,1.0,90.8,21
109198,2020-06-30,종로구,17.7,1.4,0.5,90.5,22


In [36]:
weather_final.gu.value_counts()

강북구     4368
송파구     4368
강남구     4368
성동구     4368
영등포구    4368
종로구     4368
강동구     4368
마포구     4368
도봉구     4368
용산구     4368
동작구     4368
양천구     4368
구로구     4368
서초구     4368
중구      4368
은평구     4368
성북구     4368
강서구     4368
금천구     4368
노원구     4368
중랑구     4368
광진구     4368
서대문구    4368
동대문구    4368
관악구     4368
Name: gu, dtype: int64

# 조인키 생성
- 지하철 데이터 셋을 기준으로 인구 데이터 셋과 기상 데이터 셋을 결합시켜 줄 것이므로, 이를 위한 조인키가 필요하다.
- 따라서 지하철 데이터 셋과 기상 데이터 셋을 결합하기 위한 조인키 1개를 만들어주도록 하겠다.
    - 지하철 데이터 셋과 기상 데이터 셋을 결합하기 위한 조인키(weather_join_key)
        - ex) "2018-01-01_중구"

In [37]:
# datetime to string
weather_final['date'] = weather_final['date'].astype(str)

# 지하철 데이터와 결합할 때 사용할 조인키
weather_final['weather_join_key'] = weather_final['date'] + '_' + weather_final['gu']
weather_final['weather_join_key']

0         2020-01-01_성동구
1         2020-01-01_성동구
2         2020-01-01_성동구
3         2020-01-01_성동구
4         2020-01-01_성동구
               ...      
109195    2020-06-30_종로구
109196    2020-06-30_종로구
109197    2020-06-30_종로구
109198    2020-06-30_종로구
109199    2020-06-30_종로구
Name: weather_join_key, Length: 109200, dtype: object

# 결측치 처리
- humidity 변수의 경우, 결측치가 너무 많기 때문에 분석 대상에서 제외시키도록 하겠다.
    - 즉, humidity 변수는 그냥 제거해주겠다.
- temp, windspeed, rain 변수의 경우에는 결측치들을 앞/뒤 값의 평균 값으로 대체해주겠다.
    - 만약 앞/뒤 값 중, 결측치가 하나라도 존재해서 대체가 안 되는 경우에는 전체 평균 값으로 대체해주도록 하겠다.

In [38]:
weather_final.isnull().sum()

date                    0
gu                      0
temp                  375
windspeed             350
rain                  496
humidity            43919
hour                    0
weather_join_key        0
dtype: int64

In [39]:
# humidity 변수 제거
weather_final.drop(['humidity'], axis = 1, inplace = True)

In [40]:
# temp 변수에서 결측치들이 존재하는 index 추출
temp_null_idx = weather_final[weather_final['temp'].isnull()].index
# windspeed 변수에서 결측치들이 존재하는 index 추출
windspeed_null_idx = weather_final[weather_final['windspeed'].isnull()].index
# rain 변수에서 결측치들이 존재하는 index 추출
rain_null_idx = weather_final[weather_final['rain'].isnull()].index

In [41]:
# temp 변수 결측치 앞/뒤 값의 평균 값으로 대체
for idx in tqdm(temp_null_idx):
    temp_fillna = round(weather_final.loc[idx - 1, 'temp'] + weather_final.loc[idx + 1, 'temp'] / 2, 1)
    weather_final.loc[weather_final.index == idx, 'temp'] = temp_fillna

# windspeed 변수 결측치 앞/뒤 값의 평균 값으로 대체
for idx in tqdm(windspeed_null_idx):
    windspeed_fillna = round(weather_final.loc[idx - 1, 'windspeed'] + weather_final.loc[idx + 1, 'windspeed'] / 2, 1)
    weather_final.loc[weather_final.index == idx, 'windspeed'] = windspeed_fillna
    
# rain 변수 결측치 앞/뒤 값의 평균 값으로 대체
for idx in tqdm(rain_null_idx):
    rain_fillna = round(weather_final.loc[idx - 1, 'rain'] + weather_final.loc[idx + 1, 'rain'] / 2, 1)
    weather_final.loc[weather_final.index == idx, 'rain'] = rain_fillna

In [42]:
weather_final.isnull().sum()

date                  0
gu                    0
temp                331
windspeed           301
rain                477
hour                  0
weather_join_key      0
dtype: int64

- 위 결과를 보면, 여전히 결측치가 존재한다.
    - 즉, 앞/뒤 값 중 하나라도 결측인 값이 존재했다는 의미이다.
- 따라서 이 경우들에 대해서는 전체 평균 값으로 대체해주도록 하겠다.

In [43]:
# temp 변수 전체 평균 값
temp_total_mean = round(np.mean(weather_final['temp']), 1)
# windspeed 변수 전체 평균 값
windspeed_total_mean = round(np.mean(weather_final['windspeed']), 1)
# rain 변수 전체 평균 값
rain_total_mean = round(np.mean(weather_final['rain']), 1)

weather_final['temp'].fillna(temp_total_mean, inplace = True)
weather_final['windspeed'].fillna(windspeed_total_mean, inplace = True)
weather_final['rain'].fillna(rain_total_mean, inplace = True)

In [44]:
weather_final.isnull().sum()

date                0
gu                  0
temp                0
windspeed           0
rain                0
hour                0
weather_join_key    0
dtype: int64

# 불필요한 변수들 제거
- gu 변수는 분석에 사용하지 않을 것이므로 제거해주겠다.

In [45]:
weather_final.columns

Index(['date', 'gu', 'temp', 'windspeed', 'rain', 'hour', 'weather_join_key'], dtype='object')

In [46]:
weather_final.drop(['gu'], axis = 1, inplace = True)

In [47]:
weather_final

,date,temp,windspeed,rain,hour,weather_join_key
0,2020-01-01,-4.4,1.1,0.0,0,2020-01-01_성동구
1,2020-01-01,-4.2,1.2,0.0,1,2020-01-01_성동구
2,2020-01-01,-4.0,1.4,0.0,2,2020-01-01_성동구
3,2020-01-01,-4.1,1.4,0.0,3,2020-01-01_성동구
4,2020-01-01,-4.1,1.5,0.0,4,2020-01-01_성동구
...,...,...,...,...,...,...
109195,2020-06-30,19.6,4.7,0.0,19,2020-06-30_종로구
109196,2020-06-30,19.3,4.4,0.0,20,2020-06-30_종로구
109197,2020-06-30,18.0,0.4,1.0,21,2020-06-30_종로구
109198,2020-06-30,17.7,1.4,0.5,22,2020-06-30_종로구


In [48]:
weather_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109200 entries, 0 to 109199
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   date              109200 non-null  object 
 1   temp              109200 non-null  float64
 2   windspeed         109200 non-null  float64
 3   rain              109200 non-null  float64
 4   hour              109200 non-null  int64  
 5   weather_join_key  109200 non-null  object 
dtypes: float64(3), int64(1), object(2)
memory usage: 5.0+ MB


# 최종 생성된 데이터 셋을 csv 파일로 내보내기

In [49]:
new_filepath = 'C:/Users/Brian/Desktop/Data Science/Projects/Playdata/dataset/made_by_전처리/test'

In [50]:
weather_final.to_csv('{}/weather_final.csv'.format(new_filepath), index = False)